## Deep Boltzman Machine

Data-dependent expectation $\mathbb{E}_{P_{data}}$ - $P_{data}(h, v; \theta) = p(h|v; \theta) P_{data}(v)$, with 
$P_{data}(v) = {1 \over N} \sum\limits_n \delta(v - v_n) $  
Model-dependent expectation $\mathbb{E}_{P_{model}}$ - $P_{model} = p(h|v; \theta) p(v; \theta)$

### Estimating the model-dependent expectations using Persistent Markov Chains  
* Stochatistc Approximation Procedure (Stochastic Maximum Likelihood)  
    1. Given $X_t$, sample a new state $X_{t+1}$ from a transition operator $T^θ_t(X_{t+1}; X_t)$ that leaves $p^θ_t$ invariant.  
    2. Obtain a new parameter $θ_{t+1}$ by using the expectation with respect to $X_{t+1}$ instead of the intractable model’s expectation.  
* sufficient condition for a.s. convergence - the learning rate has to decrease with time:
    * $\sum\limits^∞_{t=0} α_t = ∞$ and $\sum\limits^∞_{t=0} α^2_t < ∞$.
    * e.g. $α_t = 1/t$
* many parallel persistent chains
* a “fantasy” particle is the current state in each of these chains 

### Estimating the data-dependent expectations using variational lower bound  
* posterior $ p(h|v) $ approximated by $ q(h|v; \mu) $  
* the variationial lower bound of the log-likelihood:  
    $$ \log p(v; \theta) = \log \sum_h p(v|h; \theta) p(h)  \geq \sum_h q(h|v; \mu) \log p(v, h; \theta) + \mathcal{H}(q) \\
    = \mathbb{E}_{q(h|v)} \log  p(v,h; \theta) - KL\left[q(h|v) || p(h|v) \right] 
    = \log p(v; \theta) - KL[q(h|v; \mu)||p(h|v; \theta)]$$  
* a simple mean-field approximation $ q(h; \mu) = \prod^P_{j=1} q(h_i) = \prod^P_{j=1} \mu_i^{h_i}$, where $q(h_i = 1) = \mu_i$ and
$P$ is a number of hidden units
* optimizing the bound after plugging-in above mf:
    $$ \log p(v; \theta) \geq {1 \over 2} \sum_{i,k} L_{ik}v_iv_k +
        {1 \over 2} \sum_{j,m} J_{j,m} \mu_j \mu_m +
        \sum_{i,j} W_{i,j} v_i \mu_j - \log Z(\theta) +
        \sum_j [\mu_j \log \mu_j + (1 - \mu_j ) \log (1 - \mu_j )] $$
where $L=J=0$ for a Restricted BM:
    $$  \log p(v; \theta) \geq         
        \sum_{i,j} W_{i,j} v_i \mu_j + \sum_j [\mu_j \log \mu_j + (1 - \mu_j ) \log (1 - \mu_j )] - \log Z(\theta) $$
yields the mean-field fixed-point equations for updating of the variational parameter  
 $$ \mu_j \leftarrow \sigma \left( \sum_i W_{i,j}v_i + \sum_{m \backslash j} J_{m,j} \mu_m \right)$$
    * the 2nd term in the inequality represents the cost to maximize, where 
    $$Z(\theta) = \sum\limits_{v, h} p(v | h; \theta) * q(h)$$ 
    corresponds to the expected model-dependent energy and the rest of substracted terms corresponds to the expected data-dependent energy 

### Algorithm:  
1. randomly initialize parameters $\theta_0$ and fantasy particles $\left\{\left(\tilde{v}_0^i, \tilde{h}_0^i\right) \right\}_{i=1}^M$  
2. for number of iterations:  
    1. model-dependent inference part, i.e. for each $v^n$:
        * randomly initialize $\mu$ and run mean-field updates until convergence to obtain $\mu^n=\mu$
    2. data-dependent inference part - for each fantasy particle:  
        * run a Gibbs sampler to obtain a new state $\left(\tilde{v}_{t+1}^i, \tilde{h}_{t+1}^i\right)$ given the previous one $\left(\tilde{v}_t^i, \tilde{h}_{t}^i\right)$
3. update the parameters $\theta_{t+1}$ using the gradients of the difference between the model-dependent and data-dependent (free) energies  
4. decrease the learning rate $\alpha_t$

### Parameter initialization:  
    * Weight doubling to eliminate the double-counting problem when top-down and bottom-up influences are subsequently combined - double the input and tie the visibleto-hidden weights for lower-level layers

Reference:  
http://www.cs.toronto.edu/~fritz/absps/dbm.pdf  
http://www.cs.toronto.edu/~rsalakhu/papers/dbmrec.pdf

### Example: RBM as a single layer DBM  
$$ p(h_i = 1|v) = \sigma(v^T W_{:,i} + b_i) $$  
$$ p(v_j = 1|h) = \sigma(W_{j,:} h + b_j)$$

## Multi-prediction DBM

* a vector containing all observed variables:
    * unsupervised learning $O = v$
    * supervised learning $O = [v, y]^T$, where y is available only during training
    
* a training set $D = \{ O_i \}_{i=1}^N$
* a sequence of subsets of the possible indices of O, i.e. $S = (S_j)_{j}$, where $S_j = \{k \; | \; O_{j, k} \} $
* $Q_i$ a variational (mean-field) approximation to the joint of $P(O_{S_i}, h | O_{-S_i})$, such that
$$ Q_i(O_{S_i} , h) = \mathop{\arg \min}_{Q} KL(Q(O_{S_i}, h) \| P(O_{S_i}, h | O_{−S_i}))$$
* constraining Q to be factorial, i.e. $$ Q(O_{S_i}, h; \mu) = \prod^P_{j=1} q(h_j; \mu_j) = \prod^P_{j=1} \mu_j^{h_j}$$ where
$$ \mu_j \leftarrow \sigma \left( \sum_i W_{i,j}v_i + \sum_{m \backslash j} J_{m,j} \mu_m \right) $$

Objective:
$$ J(D, \theta) = - \sum_{O \in D} \sum_i \log Q_i(O_{S_i}) $$

0. important to preserve the order of sampling 
1. sample different subsets S_i n-times (large number)
2. run n mean-field fixed point equations as recurrent nets for fixed number of steps
3. construct n predictions of distribution Q
4. take geometric mean and renormalize to estimate final distribution - for efficient approximation average at each step of
inference, instead of the entire inference process for correct geometric mean

### The multi-inference trick
* each RNN we average over solves a different inference problem
* half of the cases, vi is observed, i.e. contributes $v_iW_{ij}$ to $h_j$’s input 
* other half of the cases, vi is inferred
* for $r_i$ representing the mean field estimate of $v_i$, then contributes $r_iW_{ij}$ to $h_j$’s input, replacing references to v with $0.5(v + r)$, where r is updated at each mean field iteration
* a good way to incorporate information from many RNNs trained in slightly different ways
* can also be understood as including an input denoising step built into the inference
* in practice, multi-inference used for the network trained without running mean field up to convergence, while for a net trained with converged mean field, each RNN is just solving an optimization problem in a graphical model
* the multi-inference trick is mostly useful as a cheap alternative for fast training and evaluation rather than high test accuracy

### Centering  
* center each variable (hidden or visible) by an offset
* helps to train without pre-training
* alone not good for classification

In [ ]:
# initialize parameters and fantasy particles
def init_params():
    # init W_i, b_i for layer i
    pass

def init_fantasy():
    # init vis_0
    # init hid_0
    pass

In [ ]:
# model-dependent part
def init_mean_field(below=None, W=None, b=None, above=None, W_above=None):
    # initialize the current layer from the layer below by using double weights
    return T.dot(below2*W)

# mean-field fixed equations step
def mean_field_iter(below, W, b, above=None, W_above=None):
    # markov blanket for layer i
    if above is not None and W_above is not None:
        mu = T.dot(below.T, W) + T.dot(above, mu_above) + b
    else:
        
    
# iteration
 
# model-dependent energy
def model_energy(vis):
    # using samples of vis and variational pars mu
    pass

In [ ]:
# data-dependent energy
def data_energy(vis):
    # using samples of vis and hid
    pass

In [ ]:
for i in xrange(0, len(dbm.hidden_layers) - 1):
    # do double weights update for_layer_i
    if i == 0:
        H_hat.append(dbm.hidden_layers[i].mf_update(
            state_above=None,
            double_weights=True,
            state_below=dbm.visible_layer.upward_state(V),
            iter_name='0'))
    else:
        H_hat.append(dbm.hidden_layers[i].mf_update(
            state_above=None,
            double_weights=True,
            state_below=dbm.hidden_layers[
                i - 1].upward_state(H_hat[i - 1]),
            iter_name='0'))

Reference:  
http://www.cs.toronto.edu/~fritz/absps/dbm.pdf  
http://www.cs.toronto.edu/~rsalakhu/papers/dbmrec.pdf  
http://www-etud.iro.umontreal.ca/~goodfeli/mp_dbm.pdf  
http://arxiv.org/pdf/1301.3568.pdf  
https://github.com/lisa-lab/pylearn2/tree/master/pylearn2/models/dbm  
https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/models/dbm/dbm.py  
https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/tests/test_dbm_metrics.py